# Chains in LangChain

Watch:
https://learn.deeplearning.ai/langchain/lesson/4/chains 

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ : bool = load_dotenv(find_dotenv()) # read local .env file

In [6]:
llm_model: str = "gpt-3.5-turbo-0613"

In [2]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [3]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [7]:
llm: ChatOpenAI = ChatOpenAI(temperature=0.9, model=llm_model)

In [8]:
prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [9]:
chain: LLMChain = LLMChain(llm=llm, prompt=prompt)

In [11]:
product: str = "Queen Size Sheet Set"
response: str = chain.run(product)

'"RoyalRest Bedding" would be a suitable name to describe a company that makes Queen Size Sheet Sets.'

## SimpleSequentialChain

![ROUTER_CHAIN](seq_chain.png)

![ROUTER_CHAIN](simple_chain.png)

In [12]:
from langchain.chains import SimpleSequentialChain

In [13]:
llm: ChatOpenAI = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one: LLMChain = LLMChain(llm=llm, prompt=first_prompt)

In [14]:
# prompt template 2
second_prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two : LLMChain = LLMChain(llm=llm, prompt=second_prompt)

In [17]:
overall_simple_chain: SimpleSequentialChain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose = True)

In [18]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Regal Linens
Regal Linens offers luxurious and elegant bedding and home décor products, adding a touch of opulence to any space.

> Finished chain.


'Regal Linens offers luxurious and elegant bedding and home décor products, adding a touch of opulence to any space.'

## SequentialChain

![ROUTER_CHAIN](seq1_chain.png)

In [19]:
from langchain.chains import SequentialChain

In [20]:
llm: ChatOpenAI = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one: LLMChain = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [21]:
second_prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two: LLMChain = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [22]:
# prompt template 3: translate to english
third_prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three: LLMChain = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [23]:
# prompt template 4: follow up message
fourth_prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four: LLMChain = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [24]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain: SequentialChain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [25]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0613 in organization org-9oP4S86HpNNbJMudY026ckbW on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0613 in organization org-9oP4S86HpNNbJMudY026ckbW on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..



> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the taste of the product, mentioning that the foam is odd and believes that it might be an old batch or counterfeit.',
 'followup_message': "Réponse de suivi: Nous sommes désolés d'apprendre que vous n'avez pas été satisfait(e) du goût de notre produit. Votre avis est précieux et nous tenons à vous assurer que nous prenons vos préoccupations très au sérieux. Nous nous efforçons constamment d'offrir une expérience gustative exceptionnelle à nos clients. \n\nConcernant la mousse étrange que vous avez mentionnée, nous tenons à préciser que notre produit est fabriqué en utilisant uniquement des ing

## Router Chain

![ROUTER_CHAIN](router_chain.png)

In [26]:
physics_template: str = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template: str = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template: str = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template: str = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [31]:
prompt_infos: list[dict[str,str]] = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [28]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [29]:
llm: ChatOpenAI = ChatOpenAI(temperature=0, model=llm_model)

In [30]:
destination_chains: dict[str, LLMChain] = {}
for p_info in prompt_infos:
    name: str = p_info["name"]
    prompt_template: str = p_info["prompt_template"]
    prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(template=prompt_template)
    chain: LLMChain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations: list[str] = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str: str = "\n".join(destinations)

In [33]:
default_prompt: ChatPromptTemplate = ChatPromptTemplate.from_template("{input}")
default_chain: LLMChain = LLMChain(llm=llm, prompt=default_prompt)

In [34]:
MULTI_PROMPT_ROUTER_TEMPLATE: str = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [35]:
router_template: str = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt: PromptTemplate = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain: LLMRouterChain = LLMRouterChain.from_llm(llm, router_prompt)

In [37]:
chain.run("What is black body radiation?")

"Black body radiation refers to the electromagnetic radiation emitted by an idealized object known as a black body. A black body is an object that absorbs all incident radiation and reflects or transmits none of it. It is a theoretical concept used to understand and model the behavior of radiation.\n\nWhen a black body is heated, it emits radiation across a wide range of wavelengths. The distribution of this radiation is characterized by Planck's law, which describes the intensity of radiation at different wavelengths and temperatures. The emitted radiation covers the entire electromagnetic spectrum, from radio waves to gamma rays.\n\nThe key characteristic of black body radiation is that its intensity and distribution depend solely on the temperature of the black body. As the temperature increases, the peak intensity of the radiation shifts towards shorter wavelengths, resulting in a change in color. For example, at lower temperatures, the radiation appears predominantly in the infrar

In [38]:
chain.run("what is 2 + 2")

'The sum of 2 + 2 is 4.'

In [39]:
chain.run("Why does every cell in our body contain DNA?")

'Every cell in our body contains DNA because DNA, or deoxyribonucleic acid, is the genetic material that carries the instructions necessary for the development, functioning, and reproduction of all living organisms. DNA contains the blueprint for building and maintaining an organism, including the synthesis of proteins, which are essential for various cellular processes.\n\nThe presence of DNA in every cell ensures that the genetic information required for the proper functioning of an organism is available in each individual cell. This is crucial because different cells in our body perform specialized functions and require specific instructions to carry out their tasks effectively.\n\nAdditionally, DNA replication ensures that each new cell produced during cell division receives an identical copy of the genetic information. This process ensures the continuity and stability of the genetic code across generations of cells.\n\nFurthermore, DNA serves as a storage medium for hereditary inf